# SYCL devices

In [ ]:
%%writefile compute.cpp
#include <sycl/sycl.hpp>
#include <vector>

int main(int argc, char **argv) {
  const int N = 2;

  std::vector<int> vectorA(N, 1);  // Vector A filled with 1s
  std::vector<int> vectorB(N, 2);  // Vector B filled with 2s
  std::vector<int> vectorC;        // Vector C initialized with no specific values
  vectorC.resize(vectorA.size());

  auto myQueue = sycl::queue{sycl::gpu_selector_v};
  std::cout << "Running on "
              << myQueue.get_device().get_info<sycl::info::device::name>()
              << "\n";

  {
      auto bufA = sycl::buffer{vectorA.data(), sycl::range{N}};
      auto bufB = sycl::buffer{vectorB.data(), sycl::range{N}};
      auto bufC = sycl::buffer{vectorC.data(), sycl::range{N}};
    
      myQueue.submit([&](sycl::handler &cgh) {

      sycl::accessor accA { bufA, cgh, sycl::read_only};
      sycl::accessor accB { bufB, cgh, sycl::read_only };
      sycl::accessor accC { bufC, cgh, sycl::write_only, sycl::no_init };

      cgh.parallel_for(N, [=](auto i) { accC[i] = accA[i] + accB[i]; });

    });
  }

  // Print out the result of vector add
  int indices[]{0, 1, 2, (static_cast<int>(vectorA.size()) - 1)};
  constexpr size_t indices_size = sizeof(indices) / sizeof(int);
  for (int i = 0; i < indices_size; i++) {
    int j = indices[i];
    if (i == indices_size - 1) std::cout << "...\n";
    std::cout << "[" << j << "]: " << vectorA[j] << " + " << vectorB[j] << " = "
              << vectorC[j] << "\n";

  }

  return 0;
}


In [ ]:
%%writefile ./run-sycl.sh

#!/bin/bash -x
source /opt/intel/oneapi/setvars.sh > /dev/null 2>&1
icpx -fsycl compute.cpp
if [ $? -eq 0 ]; then ./a.out; fi

In [ ]:
!chmod u+x ./run-sycl.sh &&./run-sycl.sh